In [109]:
import xgboost as xgb
import alpaca_trade_api as tradeapi
from finta import TA
import pickle

# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv

In [110]:
papertrading_model = xgb.XGBClassifier()
papertrading_model.load_model('xgb_clf.bst')
scaler = pickle.load(open('scaler_model.pkl','rb'))



/Users/adriencaudron/miniforge3/envs/dev310/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [111]:
feats = ['HMA_5', 'RSI_5', 'ATR_14', 'RSI_14', 'RSI_150', 'cci']

In [112]:
alpaca_api = tradeapi.REST(alpaca_api_key,alpaca_secret_key, 'https://paper-api.alpaca.markets')

In [113]:
data_trade = alpaca_api.get_crypto_bars(['BTC/USDT'], tradeapi.TimeFrame.Hour, "2023-01-01").df

In [114]:
for time in [5,14,150]:
    data_trade['RSI_'+str(time)] = TA.RSI(data_trade, time)
data_trade['ATR_14'] = TA.ATR(data_trade, 14)
data_trade['HMA_5'] = TA.HMA(data_trade, 5)
data_trade['cci'] = TA.CCI(data_trade)

In [115]:
data_trade.dtypes

close          float64
high           float64
low            float64
trade_count      int64
open           float64
volume         float64
vwap           float64
symbol          object
RSI_5          float64
RSI_14         float64
RSI_150        float64
ATR_14         float64
HMA_5          float64
cci            float64
dtype: object

In [116]:
# transform preprocessed data to numpy array
data_to_predict = scaler.transform(data_trade[feats].dropna())

In [117]:
data_to_predict[-1]

array([93.1525989 ,  1.08527707, 39.56610612,  2.13093867,  1.98656647,
        0.56584018])

In [118]:
# make predictions using the model
predictions = papertrading_model.predict(data_to_predict)

In [119]:
BUY = predictions[-1]
display(order)

Order({   'asset_class': 'crypto',
    'asset_id': 'a1733398-6acc-4e92-af24-0d0667f78713',
    'canceled_at': None,
    'client_order_id': 'a8ef9166-ff7e-4097-aa39-d7a291cb6913',
    'created_at': '2023-03-14T03:28:22.333496888Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': '5fe2651b-df89-4185-8827-abe202577682',
    'legs': None,
    'limit_price': '1674.84753',
    'notional': None,
    'order_class': '',
    'order_type': 'limit',
    'qty': '5.742122469',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'source': None,
    'status': 'pending_new',
    'stop_price': None,
    'submitted_at': '2023-03-14T03:28:22.332410218Z',
    'subtag': None,
    'symbol': 'ETH/USD',
    'time_in_force': 'gtc',
    'trail_percent': None,
    'trail_price': None,
    'type': 'limit',
    'updated_at': '2023-03-14T03:28:2

In [120]:
# Get the current price of BTC in USD
eth_usd_price = alpaca_api.get_latest_crypto_orderbook(['ETH/USD'])['ETH/USD'].asks[0].p

# Calculate the quantity of BTC to buy
usd_balance = float(alpaca_api.get_account().cash)
quantity_to_buy = usd_balance / eth_usd_price * 0.1

In [121]:
if BUY: 

## TEST to place a buy order for 500 USD worth of ETH using Alpaca API in Python:

    # Define order parameters
    symbol = 'ETHUSD'
    order_type = 'limit'
    limit_price = eth_usd_price # limit price for the buy order
    time_in_force = 'gtc'
    qty = quantity_to_buy # quantity of ETH to buy
    side = 'buy'

    # Place buy order
    try:
        order = alpaca_api.submit_order(
            symbol=symbol,
            qty=qty,
            side=side,
            type=order_type,
            time_in_force=time_in_force,
            limit_price=limit_price
        )
        print(f"Buy order for {qty} {symbol} at {limit_price} submitted successfully.")

    except Exception as e:
        print("Error submitting buy order: ", e)

else:
    # Define order parameters
    symbol = 'ETHUSD'
    order_type = 'limit'
    limit_price = eth_usd_price # limit price for the sell order
    time_in_force = 'gtc'
    qty = quantity_to_buy # quantity of ETH to sell
    qty = float(alpaca_api.get_position(symbol).qty) # quantity of ETH to sell (same as the current position)
    side = 'sell'

    # Place sell order
    try:
        order = alpaca_api.submit_order(
            symbol=symbol,
            qty=qty,
            side=side,
            type=order_type,
            time_in_force=time_in_force,
            limit_price=limit_price
        )
        print(f"Sell order for {qty} {symbol} at {limit_price} submitted successfully.")

    except Exception as e:
        print("Error submitting sell order: ", e)



APIError: position does not exist

In [ ]:
eth_usd_price

1674.84753

In [ ]:
quantity_to_buy

5.742122469204705

In [ ]:
def trade_in_alpaca(quantity, tries=0):
    if papertrading_model.predict(data_to_predict)[-1]==1 and papertrading_model.predict(data_to_predict)[-2]==0:
        try:
            alpaca_api.submit_order(symbol='BTC/USDT',qty=0.04,side="buy",type='market', time_in_force='ioc')
        except Exception as e:
            if 'insufficient balance' in str(e) and tries<=10:
                print(quantity, tries)
                trade_in_alpaca(quantity=0.9*quantity, tries=tries+1)
            else:
                print(e)

In [ ]:
trade_in_alpaca(quantity=quantity_to_buy)

In [ ]:
alpaca_api.list_positions()

[Position({   'asset_class': 'us_equity',
     'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
     'asset_marginable': False,
     'avg_entry_price': '125.25',
     'change_today': '0.0028577125008307',
     'cost_basis': '125.25',
     'current_price': '150.9',
     'exchange': 'NASDAQ',
     'lastday_price': '150.47',
     'market_value': '150.9',
     'qty': '1',
     'qty_available': '0',
     'side': 'long',
     'symbol': 'AAPL',
     'unrealized_intraday_pl': '0.43',
     'unrealized_intraday_plpc': '0.0028577125008307',
     'unrealized_pl': '25.65',
     'unrealized_plpc': '0.2047904191616766'}),
 Position({   'asset_class': 'us_equity',
     'asset_id': 'f30d734c-2806-4d0d-b145-f9fade61432b',
     'asset_marginable': False,
     'avg_entry_price': '111.43',
     'change_today': '0.0036002618372245',
     'cost_basis': '2228.6',
     'current_price': '91.99',
     'exchange': 'NASDAQ',
     'lastday_price': '91.66',
     'market_value': '1839.8',
     'qty': '20',
     'q